# faiss基于llamaindex的嵌入

为了total-llm项目的文件问答做技术验证

## 准备

In [3]:
%%time
%%capture

!pip install llama_index.core
!pip install llama-index-llms-openai-like
!pip install llama-index-embeddings-ollama
!pip install llama-index-vector-stores-faiss
!pip install faiss-gpu

CPU times: user 34.4 ms, sys: 36.9 ms, total: 71.3 ms
Wall time: 14.8 s


In [10]:
%%time

import logging
import sys

from llama_index.core import VectorStoreIndex, Document
from llama_index.core import StorageContext
from IPython.display import Markdown, display
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.llms.openai_like import OpenAILike

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 18.1 µs


In [4]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f5cde0d37f0>, id_func=<function default_id_func at 0x7f5d9c378af0>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [5]:
%%time

llm = OpenAILike(model="qwen:14b", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 120 ms, sys: 9.48 ms, total: 129 ms
Wall time: 157 ms


In [6]:
%%time

# 初始化全局 embedding 模型
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 441 ms, sys: 45.4 ms, total: 486 ms
Wall time: 633 ms


In [12]:
%%time

# 加载文件数据
with open('./books/故乡.txt', 'rb') as file:
    byte_data = file.read()

docs = [Document(text=byte_data)]

CPU times: user 0 ns, sys: 1.23 ms, total: 1.23 ms
Wall time: 1.13 ms


## faiss 索引

In [14]:
%%time

import faiss
from llama_index.vector_stores.faiss import FaissVectorStore

# bge large embedding
d = 1024
faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context
)

CPU times: user 261 ms, sys: 19.5 ms, total: 280 ms
Wall time: 1.76 s


## 基本嵌入查询

In [16]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=10
)
response = query_engine.query("闰土是谁？")
response.print_response_stream()

闰土是鲁迅先生笔下的人物，出自小说《故乡》。他是一个朴实的农村少年，与主人公鲁迅有着深厚的友情。
CPU times: user 63.4 ms, sys: 707 µs, total: 64.1 ms
Wall time: 5.29 s


In [25]:
texts = [node.text for node in response.source_nodes]
print('\n------\n'.join(texts))

我看时，不由的非常出惊，慌忙站起身，迎着走去。

　　这来的便是闰土。虽然我一见便知道是闰土，但又不是我这记忆上的闰土了。
------
我家只有一个忙月（我们这里给人做工的分三种：整年给一定人家做工的叫长工；按日给人做工的叫短工；自己也种地，只在过年过节以及收租时候来给一定人家做工的称忙月），忙不过来，他便对父亲说，可以叫他的儿子闰土来管祭器的。
------
他见人很怕羞，只是不怕我，没有旁人的时候，便和我说话，于是不到半日，我们便熟识了。

　　我们那时候不知道谈些什么，只记得闰土很高兴，说是上城之后，见了许多没有见过的东西。

　　第二日，我便要他捕鸟。他说：

　　“这不能。
------
你们先前不是哥弟称呼么？还是照旧：迅哥儿。”母亲高兴的说。

　　“阿呀，老太太真是……这成什么规矩。那时是孩子，不懂事……”闰土说着，又叫水生上来打拱，那孩子却害羞，紧紧的只贴在他背后。
------
我躺着，听船底潺潺的水声，知道我在走我的路。我想：我竟与闰土隔绝到这地步了，但我们的后辈还是一气，宏儿不是正在想念水生么。
------
“我们沙地里，潮汛要来的时候，就有许多跳鱼儿只是跳，都有青蛙似的两个脚……”

　　阿！闰土的心里有无穷无尽的希奇的事，都是我往常的朋友所不知道的。
------
的刺去，那猹却将身一扭，反从他的胯下逃走了。

　　这少年便是闰土。我认识他时，也不过十多岁，离现在将有三十年了；那时我的父亲还在世，家景也好，我正是一个少爷。那一年，我家是一件大祭祀的值年⑶。
------
“这是第五个孩子，没有见过世面，躲躲闪闪……”

　　母亲和宏儿下楼来了，他们大约也听到了声音。

　　“老太太。信是早收到了。我实在喜欢的不得了，知道老爷回来……”闰土说。

　　“阿，你怎的这样客气起来。
------
什么都有：稻鸡，角鸡，鹁鸪，蓝背……”

　　我于是又很盼望下雪。

　　闰土又对我说：

　　“现在太冷，你夏天到我们这里来。我们日里到海边捡贝壳去，红的绿的都有，鬼见怕也有，观音手⑸也有。
------
仿佛也就如此。于是我自己解释说：故乡本也如此，——虽然没有进步，也未必有如我所感的悲凉，这只是我自己心情的改变罢了，因为我这次回乡，本没有什么好心绪。

　　我这次是专为了别他而来的。


## reranker

In [26]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 2.01 s, sys: 1.67 s, total: 3.68 s
Wall time: 19.1 s


In [27]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker]
)

CPU times: user 194 µs, sys: 0 ns, total: 194 µs
Wall time: 197 µs


In [29]:
%%time

streaming_response = query_engine.query("宏儿是谁")
streaming_response.print_response_stream()
print()

宏儿是鲁迅先生作品《故乡》中的人物，他是鲁迅八岁的侄子。

CPU times: user 466 ms, sys: 14.6 ms, total: 481 ms
Wall time: 1.27 s


In [30]:
%%time

streaming_response = query_engine.query("我和宏儿是啥关系")
streaming_response.print_response_stream()
print()

我和宏儿是亲戚关系，具体可能是家人或有亲属纽带的人。

CPU times: user 487 ms, sys: 8.25 ms, total: 495 ms
Wall time: 1.65 s


In [31]:
%%time

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是小说《故乡》中的主要人物之一，他是主人公鲁迅（笔名）少年时期的朋友。闰土的名字来源于他出生的月份是闰月，五行中缺土，因此他的父亲给他取了这个名字。闰土在故事中代表了农村生活和农民阶级的形象。

CPU times: user 521 ms, sys: 23 ms, total: 544 ms
Wall time: 3.34 s


In [32]:
%%time

streaming_response = query_engine.query("闰土和水生是啥关系")
streaming_response.print_response_stream()
print()

闰土和水生可能是亲戚关系或者有其他紧密联系，但具体的关系在给定的文本中并未明确说明。

CPU times: user 484 ms, sys: 15.9 ms, total: 500 ms
Wall time: 2.07 s


In [33]:
%%time

streaming_response = query_engine.query("水生和宏儿是啥关系")
streaming_response.print_response_stream()
print()

水生和宏儿是朋友关系。宏儿被母亲鼓励去与水生交往，说明他们之间有互动和友谊。

CPU times: user 495 ms, sys: 7.65 ms, total: 503 ms
Wall time: 2.08 s


In [34]:
%%time

streaming_response = query_engine.query("闰土小时候和成年后有哪些主要变化")
streaming_response.print_response_stream()
print()

闰土小时候与成年后的变化主要体现在以下几个方面：

1. 社会角色转变：小时候，闰土是农村孩子，过着简单的生活。而成年后，他可能需要承担家庭责任和社会职责。

2. 知识视野拓宽：在鲁迅的记忆中，闰土小时候的世界相对狭窄。而成年后，闰土可能会接触到更广阔的知识领域和信息。

3. 心理成熟度提高：随着年龄的增长，闰土的心智会更加成熟，处理问题的方式也会更为理智。

4. 生活环境变化：闰土小时候生活在海边的农村，而成年后的生活环境可能因工作、家庭等原因而发生改变。

CPU times: user 618 ms, sys: 50 ms, total: 668 ms
Wall time: 6.13 s
